In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import random
import re
from collections import Counter

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud

print("✅ All libraires Imported")

In [ ]:
class SentimentAnalyzer:
    """Multi-modal sentiment analysis"""
    
    def __init__(self) -> None:
        self.vader_analyzer = SentimentIntensityAnalyzer()
        
    def clean_text(self, text):
        # Removing URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        # Removing mentions and all also
        text = re.sub(r'@\w+|#', '', text)
        text = ' '.join(text.split())
        
        return text.strip()
    
    def analyze_textblob(self, text):
        clean_text = self.clean_text(text)
        blob = TextBlob(clean_text)
        
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        if polarity > 0.1:
            sentiment = 'Positive'
        elif polarity < -0.1:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
            
        return {
            'sentiment': sentiment,
            'polarity': polarity,
            'subjectivity': subjectivity,
            'confidence': abs(polarity)
        }
        
    def analyze_vader(self, text):
        clean_text = self.clean_text(text)
        scores = self.vader_analyzer.polarity_scores(clean_text)
        
        if scores['compound'] >= 0.05:
            sentiment = 'Positive'
        elif scores['compound'] <= -0.05:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
            
        return {
            'sentiment': sentiment,
            'positive': scores['pos'],
            'neutral': scores['neu'],
            'negative': scores['neg'],
            'compound': scores['compound']
        }
        
    def analyze_combined(self, text):
        textblob_result = self.analyze_textblob(text)
        vader_result = self.analyze_vader(text)
        
        if textblob_result['sentiment'] == vader_result['sentiment']:
            final_sentiment = textblob_result['sentiment']
            confidence = (abs(textblob_result['polarity']) + abs(vader_result['compound'])) / 2
        else:
            if abs(vader_result['compound']) > abs(textblob_result['polarity']):
                final_sentiment = vader_result['sentiment']
                confidence = abs(vader_result['compound'])
            else:
                final_sentiment = textblob_result['sentiment']
                confidence = abs(textblob_result['polarity'])
        
        return {
            'sentiment': final_sentiment,
            'confidence': confidence,
            'textblob': textblob_result,
            'vader': vader_result
        }
        
    def analyze_batch(self, texts):
        results = []
        
        for text in texts:
            result = self.analyze_combined(text)
            results.append(result)
    
        return results
